In [2]:
import numpy as np 
import pandas as pd
import Bio
from Bio.pairwise2 import format_alignment
import copy

import sys 
sys.path.append('../code_general_Finkelsteinlab/')

import Process_SeqLibrary_Finkelsteinlab as preprocess

import plotting_Finkelsteinlab as pltData

%matplotlib inline 
import matplotlib.pylab as plt 

In [73]:
# to try without the working function from berouz
df=pd.DataFrame()
df['MMpos']=['1|6','1|6','7|2','8|3','19|1','15,8']
df['ABA']=[6,4,9,3,8,2]
df['Uncertainty']=[0.1,0.03,0.3,0.2,0.68,0.9]
Grouped = df.groupby('MMpos').agg(lambda x: list(x))
Grouped.reset_index(inplace=True)
Grouped


,MMpos,ABA,Uncertainty
0,"15,8",[2],[0.9]
1,19|1,[8],[0.68]
2,1|6,"[6, 4]","[0.1, 0.03]"
3,7|2,[9],[0.3]
4,8|3,[3],[0.2]


In [95]:
s='7|22'
s.split('|')
mm = np.array(s.split('|')).astype(int)
mm

array([ 7, 22])

In [74]:
MMpos=Grouped['MMpos'].tolist()
ABA=Grouped['ABA'].tolist()
Uncertainty=Grouped['Uncertainty'].tolist()

#df['df.apply('functionname',axis=1)

In [56]:
Path = '../Data_ABA_Finkelsteinlab/cas9-target-e-replicate-1-delta-abas.txt'
data = pd.read_table(Path,comment='#')
#data

In [57]:
Cas='Cas9'
on_target = 'TTTAGACGCATAAAGATGAGACGCTGG'
seq_colname = 'Sequence'
output_colnames = ['Delta ABA (kBT)', 'Uncertainty']
out_file_name = 'Cas9_target_e.csv'
out_path = '../Data_ABA_Finkelsteinlab/cas9-target-e-replicate-1-delta-abas.txt'

new_data = preprocess.process_Finkelstein_Library(data, on_target, seq_colname, output_colnames, Cas, 
                                                  Canonical_PAM = True, Mut_type = ['OT','r'], Mut_min = 0, Mut_max = 20,
                                                  out_file_name = out_file_name, out_path = out_path, nofilter = False,
                                                  save_data = True)


TypeError: ("object of type 'filter' has no len()", 'occurred at index 2')

In [3]:
# Prepare data
# function berouz to get this csv file!
data=pd.read_csv('../Data_ABA_Finkelsteinlab/cas9-target-e-replicate-1-delta-abas_Canonical_OT-r_0-20.csv')

Grouped = data.groupby('Mutation Positions').agg(lambda x: list(x))
Grouped.reset_index(inplace=True)

for i in range(len(Grouped)):
    s=Grouped['Mutation Positions'][i]
    Grouped['Mutation Positions'][i] = np.array(s.split('|')).astype(int)

MMpos=Grouped['Mutation Positions'].tolist()
ABA=Grouped['Delta ABA (kBT)'].tolist()
Uncertainty=Grouped['Uncertainty'].tolist()
# Instead of doing this we have now a function that does this, see next window.

In [4]:
import Prepare_data as Pre
path = '../Data_ABA_Finkelsteinlab/' 
filename= 'cas9-target-e-replicate-1-delta-abas_Canonical_OT-r_0-20.csv'
xdata,ydata,yerr=Pre.Prepare_Cdata(path,filename)


In [5]:
upper_bnd =  [10.0] + [10.0]*40 +  [3.0] *2
lower_bnd = [0.0] + [-10.0]*40 + [-7.0] *2
initial_guess =  [5.0] + [0.0]*40 + [0.0] *2

In [40]:
import Chisq_Finkelstein as Chi
import SimulatedAnnealing_Finkelstein_parallel as SA
import sys
sys.path.append('../code_ABA_Finkelsteinlab/')
import Calculate_ABA_Finkelsteinlab as ABA

import functools

In [34]:
gRNA_length = 20
model_ID = 'general_energies'
concentrations=[0, 0.1, 0.3, 1, 3, 10, 30, 100, 300] # in nanoMolair
reference=1 # in nanomolair


KineticModel = functools.partial(Chi.calc_Chi_square,model_id=model_ID)


In [ ]:
fit_result_file= 'fit_results.txt'
monitor_file = 'monitor.txt'
init_monitor_file='init_monitor.txt'
fit_result = SA.sim_anneal_fit(xdata=xdata,
                                ydata=ydata,
                                yerr = yerr,
                                Xstart= np.array(initial_guess),
                                lwrbnd= np.array(lower_bnd),
                                upbnd= np.array(upper_bnd),
                                model='I_am_using_multi_processing_in_stead',
                                objective_function=KineticModel,
                                on_target_function=ABA.calc_ABA,
                                Tstart=100.,             # infered from run on my computer/other runs on cluster
                                use_relative_steps=False,
                                delta=1.0,
                                tol=1E-5,
                                Tfinal=0.0,
                                adjust_factor=1.1,
                                cooling_rate=0.99,
                                N_int=1000,
                                AR_low=40,
                                AR_high=60,
                                use_multiprocessing=True,
                                nprocs=1,
                                output_file_results = fit_result_file,
                                output_file_monitor = monitor_file,
                                output_file_init_monitor=init_monitor_file
                              )